***GENERATED CODE FOR classypl PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'}], 'feature': 'LocationAbbr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationAbbr'}, {'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationAbbr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'}], 'feature': 'LocationDesc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Alabama', 'max': 'Wyoming', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationDesc'}, {'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationDesc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataSource', 'transformation_label': 'String Indexer'}], 'feature': 'DataSource', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ACS 1-Year Estimates', 'max': 'YRBSS', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataSource'}, {'feature_label': 'DataSource', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataSource')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Topic', 'transformation_label': 'String Indexer'}], 'feature': 'Topic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Alcohol', 'max': 'Tobacco', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Topic'}, {'feature_label': 'Topic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Topic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Question', 'transformation_label': 'String Indexer'}], 'feature': 'Question', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Alcohol use among youth', 'max': 'Sale of cigarette packs', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Question'}, {'feature_label': 'Question', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Question')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Response', 'transformation_label': 'String Indexer'}], 'feature': 'Response', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Response'}, {'feature_label': 'Response', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Response')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValueUnit', 'transformation_label': 'String Indexer'}], 'feature': 'DataValueUnit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '251', 'mean': '', 'stddev': '', 'min': '%', 'max': 'pack sales per capita', 'missing': '249', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValueUnit'}, {'feature_label': 'DataValueUnit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValueUnit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValueType', 'transformation_label': 'String Indexer'}], 'feature': 'DataValueType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Age-adjusted Rate', 'max': 'Number', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValueType'}, {'feature_label': 'DataValueType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValueType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValue', 'transformation_label': 'String Indexer'}], 'feature': 'DataValue', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '373', 'mean': '3094.61', 'stddev': '15977.73', 'min': '0.5', 'max': 'Category 4 - State had mixed alcohol retail licensing policies', 'missing': '127', 'distinct': '315'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValue'}, {'feature_label': 'DataValue', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValue')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValueFootnoteSymbol', 'transformation_label': 'String Indexer'}], 'feature': 'DataValueFootnoteSymbol', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '129', 'mean': '', 'stddev': '', 'min': '&', 'max': '~', 'missing': '371', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValueFootnoteSymbol'}, {'feature_label': 'DataValueFootnoteSymbol', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValueFootnoteSymbol')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DatavalueFootnote', 'transformation_label': 'String Indexer'}], 'feature': 'DatavalueFootnote', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '129', 'mean': '', 'stddev': '', 'min': 'Data not shown because of too few respondents or cases', 'max': 'No data available', 'missing': '371', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DatavalueFootnote'}, {'feature_label': 'DatavalueFootnote', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DatavalueFootnote')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Gender', 'max': 'Race/Ethnicity', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory1'}, {'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'American Indian or Alaska Native', 'max': 'White, non-Hispanic', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification1'}, {'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory2', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory2'}, {'feature_label': 'StratificationCategory2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification2', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification2'}, {'feature_label': 'Stratification2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory3', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory3'}, {'feature_label': 'StratificationCategory3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory3')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification3', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification3'}, {'feature_label': 'Stratification3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification3')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GeoLocation', 'transformation_label': 'String Indexer'}], 'feature': 'GeoLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '481', 'mean': '', 'stddev': '', 'min': 'POINT (-100.11842104899966 47.47531977900047)', 'max': 'POINT (144.793731 13.444304)', 'missing': '19', 'distinct': '54'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GeoLocation'}, {'feature_label': 'GeoLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GeoLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ResponseID', 'transformation_label': 'String Indexer'}], 'feature': 'ResponseID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ResponseID'}, {'feature_label': 'ResponseID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ResponseID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopicID', 'transformation_label': 'String Indexer'}], 'feature': 'TopicID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ALC', 'max': 'TOB', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopicID'}, {'feature_label': 'TopicID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopicID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'QuestionID', 'transformation_label': 'String Indexer'}], 'feature': 'QuestionID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ALC10_0', 'max': 'TOB10_0', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'QuestionID'}, {'feature_label': 'QuestionID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('QuestionID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValueTypeID', 'transformation_label': 'String Indexer'}], 'feature': 'DataValueTypeID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AGEADJRATE', 'max': 'NMBR', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValueTypeID'}, {'feature_label': 'DataValueTypeID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValueTypeID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategoryID1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategoryID1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'GENDER', 'max': 'RACE', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategoryID1'}, {'feature_label': 'StratificationCategoryID1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'StratificationCategoryID1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationID1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationID1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AIAN', 'max': 'WHT', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationID1'}, {'feature_label': 'StratificationID1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationID1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategoryID2', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategoryID2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategoryID2'}, {'feature_label': 'StratificationCategoryID2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'StratificationCategoryID2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationID2', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationID2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationID2'}, {'feature_label': 'StratificationID2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationID2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategoryID3', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategoryID3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategoryID3'}, {'feature_label': 'StratificationCategoryID3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'StratificationCategoryID3')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationID3', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationID3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationID3'}, {'feature_label': 'StratificationID3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationID3')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classyplHooks.ipynb
try:
	#sourcePreExecutionHook()

	cdccdik = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/CDC_CDI_2K.csv', 'filename': 'CDC_CDI_2K.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(cdccdik)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classyplHooks.ipynb
try:
	#transformationPreExecutionHook()

	classyplautofe = TransformationMain.run(cdccdik,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearStart", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2014.92", "stddev": "3.17", "min": "2008", "max": "2020", "missing": "0"}, "updatedLabel": "YearStart"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearEnd", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2015.49", "stddev": "2.84", "min": "2010", "max": "2020", "missing": "0"}, "updatedLabel": "YearEnd"}, {"transformationsData": [{"feature_label": "LocationAbbr", "transformation_label": "String Indexer"}], "feature": "LocationAbbr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationAbbr"}, {"transformationsData": [{"feature_label": "LocationDesc", "transformation_label": "String Indexer"}], "feature": "LocationDesc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Alabama", "max": "Wyoming", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationDesc"}, {"transformationsData": [{"feature_label": "DataSource", "transformation_label": "String Indexer"}], "feature": "DataSource", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ACS 1-Year Estimates", "max": "YRBSS", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataSource"}, {"transformationsData": [{"feature_label": "Topic", "transformation_label": "String Indexer"}], "feature": "Topic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Alcohol", "max": "Tobacco", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Topic"}, {"transformationsData": [{"feature_label": "Question", "transformation_label": "String Indexer"}], "feature": "Question", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Alcohol use among youth", "max": "Sale of cigarette packs", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Question"}, {"transformationsData": [{"feature_label": "Response", "transformation_label": "String Indexer"}], "feature": "Response", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Response"}, {"transformationsData": [{"feature_label": "DataValueUnit", "transformation_label": "String Indexer"}], "feature": "DataValueUnit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "251", "mean": "", "stddev": "", "min": "%", "max": "pack sales per capita", "missing": "249", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValueUnit"}, {"transformationsData": [{"feature_label": "DataValueType", "transformation_label": "String Indexer"}], "feature": "DataValueType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Age-adjusted Rate", "max": "Number", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValueType"}, {"transformationsData": [{"feature_label": "DataValue", "transformation_label": "String Indexer"}], "feature": "DataValue", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "373", "mean": "3094.61", "stddev": "15977.73", "min": "0.5", "max": "Category 4 - State had mixed alcohol retail licensing policies", "missing": "127", "distinct": "315"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValue"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DataValueAlt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "371", "mean": "3094.61", "stddev": "15977.73", "min": "0.5", "max": "162877.0", "missing": "129"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DataValueAlt"}, {"transformationsData": [{"feature_label": "DataValueFootnoteSymbol", "transformation_label": "String Indexer"}], "feature": "DataValueFootnoteSymbol", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "129", "mean": "", "stddev": "", "min": "&", "max": "~", "missing": "371", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValueFootnoteSymbol"}, {"transformationsData": [{"feature_label": "DatavalueFootnote", "transformation_label": "String Indexer"}], "feature": "DatavalueFootnote", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "129", "mean": "", "stddev": "", "min": "Data not shown because of too few respondents or cases", "max": "No data available", "missing": "371", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DatavalueFootnote"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LowConfidenceLimit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "159", "mean": "19.43", "stddev": "15.41", "min": "0.1", "max": "74.8", "missing": "341"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LowConfidenceLimit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HighConfidenceLimit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "159", "mean": "27.61", "stddev": "20.59", "min": "1.6", "max": "130.4", "missing": "341"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "HighConfidenceLimit"}, {"transformationsData": [{"feature_label": "StratificationCategory1", "transformation_label": "String Indexer"}], "feature": "StratificationCategory1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Gender", "max": "Race/Ethnicity", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory1"}, {"transformationsData": [{"feature_label": "Stratification1", "transformation_label": "String Indexer"}], "feature": "Stratification1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "American Indian or Alaska Native", "max": "White, non-Hispanic", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification1"}, {"transformationsData": [{"feature_label": "StratificationCategory2", "transformation_label": "String Indexer"}], "feature": "StratificationCategory2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory2"}, {"transformationsData": [{"feature_label": "Stratification2", "transformation_label": "String Indexer"}], "feature": "Stratification2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification2"}, {"transformationsData": [{"feature_label": "StratificationCategory3", "transformation_label": "String Indexer"}], "feature": "StratificationCategory3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory3"}, {"transformationsData": [{"feature_label": "Stratification3", "transformation_label": "String Indexer"}], "feature": "Stratification3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification3"}, {"transformationsData": [{"feature_label": "GeoLocation", "transformation_label": "String Indexer"}], "feature": "GeoLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "481", "mean": "", "stddev": "", "min": "POINT (-100.11842104899966 47.47531977900047)", "max": "POINT (144.793731 13.444304)", "missing": "19", "distinct": "54"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GeoLocation"}, {"transformationsData": [{"feature_label": "ResponseID", "transformation_label": "String Indexer"}], "feature": "ResponseID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ResponseID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "28.33", "stddev": "17.5", "min": "1", "max": "78", "missing": "0"}, "updatedLabel": "LocationID"}, {"transformationsData": [{"feature_label": "TopicID", "transformation_label": "String Indexer"}], "feature": "TopicID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ALC", "max": "TOB", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopicID"}, {"transformationsData": [{"feature_label": "QuestionID", "transformation_label": "String Indexer"}], "feature": "QuestionID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ALC10_0", "max": "TOB10_0", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "QuestionID"}, {"transformationsData": [{"feature_label": "DataValueTypeID", "transformation_label": "String Indexer"}], "feature": "DataValueTypeID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AGEADJRATE", "max": "NMBR", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValueTypeID"}, {"transformationsData": [{"feature_label": "StratificationCategoryID1", "transformation_label": "String Indexer"}], "feature": "StratificationCategoryID1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "GENDER", "max": "RACE", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategoryID1"}, {"transformationsData": [{"feature_label": "StratificationID1", "transformation_label": "String Indexer"}], "feature": "StratificationID1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AIAN", "max": "WHT", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationID1"}, {"transformationsData": [{"feature_label": "StratificationCategoryID2", "transformation_label": "String Indexer"}], "feature": "StratificationCategoryID2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategoryID2"}, {"transformationsData": [{"feature_label": "StratificationID2", "transformation_label": "String Indexer"}], "feature": "StratificationID2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationID2"}, {"transformationsData": [{"feature_label": "StratificationCategoryID3", "transformation_label": "String Indexer"}], "feature": "StratificationCategoryID3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategoryID3"}, {"transformationsData": [{"feature_label": "StratificationID3", "transformation_label": "String Indexer"}], "feature": "StratificationID3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationID3"}]}))

	#transformationPostExecutionHook(classyplautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classyplHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classyplautofe, ["YearStart", "YearEnd", "DataValueAlt", "LowConfidenceLimit", "HighConfidenceLimit", "LocationID", "LocationAbbr_stringindexer", "LocationDesc_stringindexer", "DataSource_stringindexer", "Topic_stringindexer", "Question_stringindexer", "Response_stringindexer", "DataValueType_stringindexer", "DataValue_stringindexer", "DataValueFootnoteSymbol_stringindexer", "DatavalueFootnote_stringindexer", "StratificationCategory1_stringindexer", "Stratification1_stringindexer", "StratificationCategory2_stringindexer", "Stratification2_stringindexer", "StratificationCategory3_stringindexer", "Stratification3_stringindexer", "GeoLocation_stringindexer", "ResponseID_stringindexer", "TopicID_stringindexer", "QuestionID_stringindexer", "DataValueTypeID_stringindexer", "StratificationCategoryID1_stringindexer", "StratificationID1_stringindexer", "StratificationCategoryID2_stringindexer", "StratificationID2_stringindexer", "StratificationCategoryID3_stringindexer", "StratificationID3_stringindexer"], "DataValueUnit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

